In [1]:
import clrs

2024-05-08 16:10:58.142960: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/rgildin/miniconda3/envs/py310_tacticai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sampler, spec = clrs.build_sampler(
    name='bfs',
    seed=42,
    num_samples=1000,
    length=16)

In [8]:
sampler._outputs

[DataPoint(name="pi",	location=node,	type=pointer,	data=Array(1000, 16))]

In [3]:
sampler

In [ ]:
from clrs.examples.utils import *

## Main

In [ ]:
if FLAGS.hint_mode == 'encoded_decoded':
    encode_hints = True
    decode_hints = True
elif FLAGS.hint_mode == 'decoded_only':
    encode_hints = False
    decode_hints = True
elif FLAGS.hint_mode == 'none':
    encode_hints = False
    decode_hints = False
else:
    raise ValueError('Hint mode not in {encoded_decoded, decoded_only, none}.')

train_lengths = [int(x) for x in FLAGS.train_lengths]

rng = np.random.RandomState(FLAGS.seed)
rng_key = jax.random.PRNGKey(rng.randint(2**32))

In [ ]:
# Create samplers
(train_samplers,
val_samplers, val_sample_counts,
test_samplers, test_sample_counts,
spec_list) = create_samplers(rng, train_lengths)

In [ ]:
train_lengths

# create_samplers

In [ ]:
# def create_samplers(rng, train_lengths: List[int]):
"""Create all the samplers."""
train_samplers = []
val_samplers = []
val_sample_counts = []
test_samplers = []
test_sample_counts = []
spec_list = []

In [ ]:
filename = '/tmp/CLRS30/CLRS30_v1.0.0/clrs_dataset/activity_selector_test/1.0.0/clrs_dataset-test.tfrecord-00000-of-00001'
data = tf.io.read_file(filename)

In [ ]:
import tensorflow_datasets as tfds
folder = '/tmp/CLRS30/CLRS30_v1.0.0/clrs_dataset/activity_selector_test'
dataset = tfds.load(folder)